In [39]:
num_satpam = int(input("Silakan hitung 3 digit terakhir dari nidn anda dan tuliskan hasilnya: "))
if num_satpam < 9:
    num_satpam = num_satpam + 8

Silakan hitung 3 digit terakhir dari nidn anda dan tuliskan hasilnya: 2


In [40]:
print(f"Universitas Islam Riau memiliki {num_satpam} satpam yang bekerja beberapa shift. UIR perlu menjadwalkan satpam yang cukup untuk setiap shift harian. Biasanya, jadwal akan memiliki kendala, yaitu:\n1. Setiap hari dibagi menjadi 9 shift untuk menjaga di 9 fakultas berbeda.\n2. Setiap hari, setiap shift ditugaskan untuk satu satpam, dan tidak ada satpam yang bekerja lebih dari satu shift.\n3. Setiap satpam ditugaskan untuk setidaknya dua shift selama periode 7 hari.\nUIR kesulitan untuk menyusun jadwal yang memenuhi semua kendala dan meminta bantuan Anda untuk menyusun jadwal yang optimal.")

Universitas Islam Riau memiliki 10 satpam yang bekerja beberapa shift. UIR perlu menjadwalkan satpam yang cukup untuk setiap shift harian. Biasanya, jadwal akan memiliki kendala, yaitu:
1. Setiap hari dibagi menjadi 9 shift untuk menjaga di 9 fakultas berbeda.
2. Setiap hari, setiap shift ditugaskan untuk satu satpam, dan tidak ada satpam yang bekerja lebih dari satu shift.
3. Setiap satpam ditugaskan untuk setidaknya dua shift selama periode 7 hari.
UIR kesulitan untuk menyusun jadwal yang memenuhi semua kendala dan meminta bantuan Anda untuk menyusun jadwal yang optimal.


In [44]:
from ortools.sat.python import cp_model

In [54]:
# Data (20 poin)
def main():
    num_nurses = num_satpam
    num_shifts = 3
    num_days = 7
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)
    shift_requests = [[[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                      [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0],
                       [0, 0, 0], [0, 0, 1]],
                      [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0],
                       [0, 1, 0], [0, 0, 0]],
                      [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0],
                       [1, 0, 0], [0, 0, 0]],
                      [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0],
                       [0, 1, 0], [0, 0, 0]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]]]



In [59]:
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))



NameError: name 'all_nurses' is not defined

In [57]:
# Creates shift variables. (5 poin)
# shifts[(n, d, s)]: satpam 'n' works shift 's' on day 'd'.
 shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))


IndentationError: unexpected indent (680939240.py, line 3)

In [22]:
# CONSTRAINT 1 (5 poin)
# Each shift is assigned to exactly one satpam in the schedule period.
    for d in all_days:
        for s in all_shifts:
            model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)


In [23]:
# CONSTRAINT 2 (5 poin)
# Each satpam works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

In [24]:
# Try to distribute the shifts evenly, so that each satpam works (10 poin)
# min_shifts_per_satpam shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of satpam, some satpam will
# be assigned one more shift.
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    if num_shifts * num_days % num_nurses == 0:
        max_shifts_per_nurse = min_shifts_per_nurse
    else:
        max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        num_shifts_worked = 0
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked += shifts[(n, d, s)]
        model.Add(min_shifts_per_nurse <= num_shifts_worked)
        model.Add(num_shifts_worked <= max_shifts_per_nurse)

In [25]:
# Creates the solver and solve. (2 poin)
    # pylint: disable=g-complex-comprehension
    model.Maximize(
        sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_nurses
            for d in all_days for s in all_shifts))
    solver = cp_model.CpSolver()
    status = solver.Solve(model)


No assignment is possible.


In [27]:
# Enumerate all solutions. (27 poin)
if status == cp_model.OPTIMAL:
        print('Solution:')
        for d in all_days:
            print('Day', d)
            for n in all_nurses:
                for s in all_shifts:
                    if solver.Value(shifts[(n, d, s)]) == 1:
                        if shift_requests[n][d][s] == 1:
                            print('Satpam', n, 'works shift', s, '(requested).')
                        else:
                            print('Satpam', n, 'works shift', s,
                                  '(not requested).')
            print()
        print(f'Number of shift requests met = {solver.ObjectiveValue()}',
              f'(out of {num_nurses * min_shifts_per_nurse})')
    else:
        print('No optimal solution found !')

ModuleNotFoundError: No module named 'docplex'

In [ ]:
  # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())

In [53]:
"""Nurse scheduling problem with shift requests."""
from ortools.sat.python import cp_model


def main():
    # This program tries to find an optimal assignment of nurses to shifts
    # (3 shifts per day, for 7 days), subject to some constraints (see below).
    # Each nurse can request to be assigned to specific shifts.
    # The optimal assignment maximizes the number of fulfilled shift requests.
    num_nurses = num_satpam
    num_shifts = 3
    num_days = 7
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)
    shift_requests = [[[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                      [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0],
                       [0, 0, 0], [0, 0, 1]],
                      [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0],
                       [0, 1, 0], [0, 0, 0]],
                      [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0],
                       [1, 0, 0], [0, 0, 0]],
                      [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0],
                       [0, 1, 0], [0, 0, 0]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                     [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]]]

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Each shift is assigned to exactly one nurse in .
    for d in all_days:
        for s in all_shifts:
            model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

    # Try to distribute the shifts evenly, so that each nurse works
    # min_shifts_per_nurse shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of nurses, some nurses will
    # be assigned one more shift.
    min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    if num_shifts * num_days % num_nurses == 0:
        max_shifts_per_nurse = min_shifts_per_nurse
    else:
        max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        num_shifts_worked = 0
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked += shifts[(n, d, s)]
        model.Add(min_shifts_per_nurse <= num_shifts_worked)
        model.Add(num_shifts_worked <= max_shifts_per_nurse)

    # pylint: disable=g-complex-comprehension
    model.Maximize(
        sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_nurses
            for d in all_days for s in all_shifts))

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print('Solution:')
        for d in all_days:
            print('Day', d)
            for n in all_nurses:
                for s in all_shifts:
                    if solver.Value(shifts[(n, d, s)]) == 1:
                        if shift_requests[n][d][s] == 1:
                            print('Satpam', n, 'Jadwal shift', s, '(requested).')
                        else:
                            print('Satpam', n, 'Jadwal shift', s,
                                  '(not requested).')
            print()
        print(f'Number of shift requests met = {solver.ObjectiveValue()}',
              f'(out of {num_nurses * min_shifts_per_nurse})')
    else:
        print('No optimal solution found !')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())


if __name__ == '__main__':
    main()

Solution:
Day 0
Satpam 2 Jadwal shift 1 (requested).
Satpam 7 Jadwal shift 0 (not requested).
Satpam 9 Jadwal shift 2 (requested).

Day 1
Satpam 2 Jadwal shift 1 (requested).
Satpam 4 Jadwal shift 2 (requested).
Satpam 6 Jadwal shift 0 (not requested).

Day 2
Satpam 3 Jadwal shift 0 (requested).
Satpam 4 Jadwal shift 1 (requested).
Satpam 8 Jadwal shift 2 (not requested).

Day 3
Satpam 1 Jadwal shift 1 (requested).
Satpam 2 Jadwal shift 0 (requested).
Satpam 9 Jadwal shift 2 (not requested).

Day 4
Satpam 0 Jadwal shift 2 (requested).
Satpam 1 Jadwal shift 0 (requested).
Satpam 8 Jadwal shift 1 (not requested).

Day 5
Satpam 3 Jadwal shift 0 (requested).
Satpam 5 Jadwal shift 1 (requested).
Satpam 6 Jadwal shift 2 (not requested).

Day 6
Satpam 0 Jadwal shift 2 (requested).
Satpam 5 Jadwal shift 0 (not requested).
Satpam 7 Jadwal shift 1 (not requested).

Number of shift requests met = 13.0 (out of 20)

Statistics
  - conflicts: 2
  - branches : 421
  - wall time: 0.049230 s
